# Importing Libraries

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
!pip install pandas-profiling
!pip install ipywidgets
import pandas_profiling as pp
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns

# Loading Dataset

In [34]:
## Read in the database and csv file
file_path = "./419ProjectDataset.csv"
df = pd.read_csv(file_path, index_col='ID')

#df

In [35]:
## Reformat the dataframe

## Rename column names to shorten their names
df = df.rename(
    columns={
        "Research Paper Intensity": "ori_i",
        "Participant 1 Classified Emotion": "p1_e",
        "Participant 1 Classified Intensity" : "p1_i",
        "Participant 2 Classified Emotion": "p2_e",
        "Participant 2 Classified Intensity": "p2_i"
    }
)

#For easy reference later
emotion_cols = ['p1_e', 'p2_e']
intensity_cols = ['p1_i', 'p2_i']

In [36]:
## Finding the number of similar annotations between p1_e and p1_2
df[df['p1_e'] == 'displeased']['p2_e'].value_counts()
# displeased : 88
# depressed : 67
# hurt : 12
# lonely : 12
# grief : 10
# guilty : 6

displeased    88
depressed      6
hurt           6
lonely         2
guilty         1
Name: p2_e, dtype: int64

In [37]:
df[df['p1_e'] == df['p2_e']]

,Tweet,ori_i,p1_e,p1_i,p2_e,p2_i
ID,,,,,,
0,Depression sucks! #depression,0.958,depressed,5,depressed,4
3,Im think ima lay in bed all day and sulk. Life...,0.934,depressed,4,depressed,4
4,So when I try I fail... and when I don't try.....,0.917,displeased,3,displeased,4
5,my life in one word is depressing,0.917,depressed,4,depressed,4
6,Panic attacks are the worst. Feeling really si...,0.917,depressed,3,depressed,5
...,...,...,...,...,...,...
266,"Like, 'loon attic'. Luckily it quickly sunk in...",0.375,guilty,2,guilty,2
267,I'm too sober time to get shit faced,0.360,depressed,4,depressed,4
269,"Pops are joyless, soulless toys which look nea...",0.354,displeased,3,displeased,3


In [38]:
df_new = df[df['p1_e'] == df['p2_e']]
df_new.head()

,Tweet,ori_i,p1_e,p1_i,p2_e,p2_i
ID,,,,,,
0,Depression sucks! #depression,0.958,depressed,5,depressed,4
3,Im think ima lay in bed all day and sulk. Life...,0.934,depressed,4,depressed,4
4,So when I try I fail... and when I don't try.....,0.917,displeased,3,displeased,4
5,my life in one word is depressing,0.917,depressed,4,depressed,4
6,Panic attacks are the worst. Feeling really si...,0.917,depressed,3,depressed,5


In [39]:
# Removing Stopwords
english_stop_words = set(stopwords.words('english'))

# tf-idf Embedding

In [58]:
## Applying tf-idf - To Map tweets to numerical vetors
vectorizer = TfidfVectorizer(stop_words = english_stop_words)
X = vectorizer.fit_transform(df_new['Tweet'])
X

<195x1053 sparse matrix of type '<class 'numpy.float64'>'
	with 1697 stored elements in Compressed Sparse Row format>

In [41]:
df_with_features = pd.DataFrame(X.todense())
df_with_features

,0,1,2,3,4,5,6,7,8,9,...,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df_with_features['label'] = df_new['p1_e'].values
df_with_features

,0,1,2,3,4,5,6,7,8,9,...,1044,1045,1046,1047,1048,1049,1050,1051,1052,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,depressed
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,depressed
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,displeased
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,depressed
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,depressed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,guilty
191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,depressed
192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,displeased
193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,guilty


In [43]:
## we don't use it in this code though (in SVM)
## Create pca object
pca = PCA(n_components = 100)
df_dimension_reduced = pd.DataFrame(pca.fit_transform(df_with_features.drop(['label'], axis = 1)))
#df_dimension_reduced.columns = ['PC1', 'PC2']
df_dimension_reduced['label'] = df_with_features['label']
df_dimension_reduced

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,label
0,-0.110971,0.769686,0.254652,0.136599,0.026551,-0.043464,0.041726,0.067399,-0.004098,0.038825,...,-0.013442,-0.016496,-0.020329,-0.030158,0.002500,0.012197,0.012761,-0.031681,-0.022587,depressed
1,-0.027725,-0.026892,-0.167820,0.122445,0.081568,-0.085124,0.150683,0.015538,-0.058814,-0.033991,...,0.004862,0.077358,-0.066435,0.002891,0.016453,0.009157,-0.048914,-0.010579,0.082723,depressed
2,-0.033752,-0.032132,-0.064252,0.102854,0.002206,0.042000,-0.120516,-0.148051,-0.057929,0.029530,...,0.069969,-0.074932,0.028465,0.179437,-0.061310,-0.029038,-0.021921,0.036180,0.052017,displeased
3,0.008336,-0.030343,-0.075931,-0.070574,-0.270129,-0.028104,0.408068,-0.106064,-0.137571,0.021603,...,0.015786,0.045733,0.037725,0.013451,-0.027632,0.074021,-0.042490,-0.021629,0.011582,depressed
4,-0.059846,0.294572,0.096014,0.008826,0.058529,0.093694,-0.050610,-0.111209,-0.034798,0.033530,...,0.047547,-0.019867,-0.000361,0.014896,0.031413,0.060164,-0.072778,-0.008982,-0.042679,depressed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,-0.036436,0.006333,-0.031959,-0.063713,-0.003990,-0.019222,-0.091929,-0.135110,0.027980,0.048553,...,0.136113,0.099250,-0.151669,0.051355,0.027543,-0.047187,0.120493,0.063174,0.019057,guilty
191,-0.048593,-0.041824,-0.013271,-0.146458,-0.218839,-0.125227,-0.066374,0.266846,0.041061,0.062086,...,-0.045307,-0.046477,0.006877,-0.050187,0.003267,-0.032472,-0.058229,0.102739,-0.032532,depressed
192,-0.021841,-0.017375,-0.007108,-0.038002,-0.021822,-0.016679,-0.016717,-0.005415,-0.009133,-0.000603,...,0.056385,0.050661,-0.049179,-0.028710,-0.065151,0.012403,-0.059433,0.047157,-0.049177,displeased
193,-0.039931,-0.045553,-0.056865,-0.057153,-0.168604,-0.096829,-0.358296,0.584383,-0.374271,0.316456,...,-0.002066,0.009346,0.001030,-0.006408,-0.000783,0.020822,0.007283,0.009199,-0.001218,guilty


# Support Vector Machine (SVM) Model

In [44]:
svm_model = SVC(kernel = 'linear') # 0.57 # it should be changed to KNN, RandomForest
#knn_model = KNeighborsClassifier(n_neighbors = 15) # 0.51
#dtree_model = DecisionTreeClassifier(max_depth = 10, random_state = 101) # 0.54 # Do better for depressed percision & displeased recall
X = df_with_features.drop(['label'], axis = 1)
y = df_with_features['label']
cross_val_predict(svm_model, X, y, cv = 5)

array(['depressed', 'depressed', 'displeased', 'depressed', 'displeased',
       'displeased', 'depressed', 'depressed', 'depressed', 'depressed',
       'depressed', 'displeased', 'displeased', 'depressed', 'displeased',
       'displeased', 'displeased', 'depressed', 'displeased',
       'displeased', 'displeased', 'displeased', 'displeased',
       'displeased', 'depressed', 'displeased', 'depressed', 'displeased',
       'displeased', 'depressed', 'depressed', 'depressed', 'displeased',
       'depressed', 'displeased', 'depressed', 'displeased', 'depressed',
       'displeased', 'displeased', 'displeased', 'displeased',
       'displeased', 'depressed', 'displeased', 'displeased',
       'displeased', 'grief', 'displeased', 'displeased', 'displeased',
       'displeased', 'displeased', 'depressed', 'displeased',
       'displeased', 'displeased', 'depressed', 'displeased',
       'displeased', 'displeased', 'displeased', 'displeased',
       'depressed', 'displeased', 'displeased'

In [45]:
#profile = pp.ProfileReport(df_dimension_reduced)
#profile.to_file('EDAReport.html')

## Create a Confusion Matrix

In [46]:
X = df_with_features.drop(['label'], axis = 1)
y = df_with_features['label']
svm_model.fit(X, y)

## Apllying 5-fold cross validation
predict_svc = pd.Series(cross_val_predict(svm_model, X, y, cv = 5))

## Generating Confusoin Matrix
conf_matrix_svc = confusion_matrix(y, predict_svc, labels = ['displeased', 'depressed', 'hurt', 'lonely', 'grief', 'guilty'])
print(conf_matrix_svc)

[[79  9  0  0  0  0]
 [37 30  0  0  0  0]
 [11  1  0  0  0  0]
 [ 8  3  0  0  1  0]
 [ 8  0  0  0  2  0]
 [ 4  1  0  0  0  1]]


In [47]:
### How come it does not show the confussion matrix?
"""
## Display the visualization of the Confusion Matrix.
fig = plt.figure(figsize = (16,6))

ax = sns.heatmap(conf_matrix_svc, annot = True, cmap = 'Blues')

ax.set_title('SVM Confusion Matrix \n');
ax.set_xlabel('\nPredicted Label')
ax.set_ylabel('True Label ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['depressed', 'displeased', 'grief', 'guilty', 'hurt', 'lonely' ])
ax.yaxis.set_ticklabels(['depressed', 'displeased', 'grief', 'guilty', 'hurt', 'lonely' ])
"""

"\n## Display the visualization of the Confusion Matrix.\nfig = plt.figure(figsize = (16,6))\n\nax = sns.heatmap(conf_matrix_svc, annot = True, cmap = 'Blues')\n\nax.set_title('SVM Confusion Matrix \n');\nax.set_xlabel('\nPredicted Label')\nax.set_ylabel('True Label ');\n\n## Ticket labels - List must be in alphabetical order\nax.xaxis.set_ticklabels(['depressed', 'displeased', 'grief', 'guilty', 'hurt', 'lonely' ])\nax.yaxis.set_ticklabels(['depressed', 'displeased', 'grief', 'guilty', 'hurt', 'lonely' ])\n"

## Report precision,  recall,  accuracy,  F1 score
[Resource](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

In [48]:
## Classification Report
target_names = ['displeased', 'depressed', 'hurt', 'lonely', 'grief', 'guilty']
print(classification_report(y, predict_svc, target_names = target_names))

              precision    recall  f1-score   support

  displeased       0.68      0.45      0.54        67
   depressed       0.54      0.90      0.67        88
        hurt       0.67      0.20      0.31        10
      lonely       1.00      0.17      0.29         6
       grief       0.00      0.00      0.00        12
      guilty       0.00      0.00      0.00        12

    accuracy                           0.57       195
   macro avg       0.48      0.29      0.30       195
weighted avg       0.54      0.57      0.51       195



/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [49]:
## Prediction
sentence = ["she has depression"]
X_test1 = vectorizer.transform(sentence)
#X_test1

svm_model.fit(X, y)
svm_model.predict(X_test1.todense())

/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


array(['depressed'], dtype=object)

# KNN Model

In [50]:
#svm_model = SVC(kernel = 'linear') # 0.57 # it should be changed to KNN, RandomForest
knn_model = KNeighborsClassifier(n_neighbors = 15) # 0.51
#dtree_model = DecisionTreeClassifier(max_depth = 10, random_state = 101) # 0.54 # Do better for depressed percision & displeased recall
X = df_with_features.drop(['label'], axis = 1)
y = df_with_features['label']
cross_val_predict(knn_model, X, y, cv = 5)

array(['depressed', 'depressed', 'displeased', 'depressed', 'displeased',
       'displeased', 'depressed', 'depressed', 'depressed', 'depressed',
       'depressed', 'displeased', 'displeased', 'depressed', 'displeased',
       'displeased', 'displeased', 'depressed', 'displeased', 'depressed',
       'displeased', 'displeased', 'depressed', 'depressed', 'depressed',
       'displeased', 'depressed', 'displeased', 'displeased', 'depressed',
       'depressed', 'displeased', 'displeased', 'displeased', 'depressed',
       'displeased', 'displeased', 'depressed', 'displeased', 'depressed',
       'displeased', 'depressed', 'displeased', 'depressed', 'displeased',
       'displeased', 'displeased', 'displeased', 'displeased',
       'displeased', 'displeased', 'displeased', 'depressed', 'depressed',
       'displeased', 'displeased', 'depressed', 'displeased',
       'displeased', 'displeased', 'displeased', 'displeased',
       'displeased', 'depressed', 'displeased', 'depressed', 'disp

In [51]:
## Apllying 5-fold cross validation
predict_svc = pd.Series(cross_val_predict(knn_model, X, y, cv = 5))

## Generating Confusoin Matrix
conf_matrix_svc = confusion_matrix(y, predict_svc, labels = ['displeased', 'depressed', 'hurt', 'lonely', 'grief', 'guilty'])
print(conf_matrix_svc)

[[66 21  0  0  1  0]
 [34 33  0  0  0  0]
 [ 6  6  0  0  0  0]
 [ 6  6  0  0  0  0]
 [ 7  3  0  0  0  0]
 [ 5  1  0  0  0  0]]


In [52]:
## Classification Report
target_names = ['displeased', 'depressed', 'hurt', 'lonely', 'grief', 'guilty']
print(classification_report(y, predict_svc, target_names = target_names))

              precision    recall  f1-score   support

  displeased       0.47      0.49      0.48        67
   depressed       0.53      0.75      0.62        88
        hurt       0.00      0.00      0.00        10
      lonely       0.00      0.00      0.00         6
       grief       0.00      0.00      0.00        12
      guilty       0.00      0.00      0.00        12

    accuracy                           0.51       195
   macro avg       0.17      0.21      0.18       195
weighted avg       0.40      0.51      0.45       195



/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [53]:
## Prediction
sentence = ["she has depression"]
X_test1 = vectorizer.transform(sentence)
#X_test1

knn_model.fit(X, y)
knn_model.predict(X_test1.todense())

/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


array(['depressed'], dtype=object)

# Decision Tree Model

In [54]:
#svm_model = SVC(kernel = 'linear') # 0.57 # it should be changed to KNN, RandomForest
#knn_model = KNeighborsClassifier(n_neighbors = 15) # 0.51
dtree_model = DecisionTreeClassifier(max_depth = 10, random_state = 101) # 0.54 # Do better for depressed percision & displeased recall
X = df_with_features.drop(['label'], axis = 1)
y = df_with_features['label']
cross_val_predict(dtree_model, X, y, cv = 5)

array(['depressed', 'displeased', 'displeased', 'displeased', 'depressed',
       'displeased', 'depressed', 'depressed', 'depressed', 'displeased',
       'depressed', 'displeased', 'displeased', 'depressed', 'displeased',
       'displeased', 'displeased', 'depressed', 'displeased', 'depressed',
       'displeased', 'displeased', 'displeased', 'depressed', 'depressed',
       'displeased', 'depressed', 'displeased', 'displeased',
       'displeased', 'depressed', 'displeased', 'displeased', 'grief',
       'displeased', 'displeased', 'displeased', 'depressed',
       'displeased', 'depressed', 'displeased', 'displeased',
       'displeased', 'depressed', 'displeased', 'displeased',
       'displeased', 'grief', 'displeased', 'displeased', 'displeased',
       'displeased', 'displeased', 'displeased', 'displeased',
       'displeased', 'hurt', 'depressed', 'displeased', 'displeased',
       'displeased', 'displeased', 'displeased', 'depressed',
       'displeased', 'displeased', 'disp

In [55]:
## Apllying 5-fold cross validation
predict_svc = pd.Series(cross_val_predict(dtree_model, X, y, cv = 5))

## Generating Confusoin Matrix
conf_matrix_svc = confusion_matrix(y, predict_svc, labels = ['displeased', 'depressed', 'hurt', 'lonely', 'grief', 'guilty'])
print(conf_matrix_svc)

[[82  2  0  0  4  0]
 [42 23  0  0  2  0]
 [10  1  0  0  1  0]
 [10  1  1  0  0  0]
 [ 9  0  0  0  1  0]
 [ 6  0  0  0  0  0]]


In [56]:
## Classification Report
target_names = ['displeased', 'depressed', 'hurt', 'lonely', 'grief', 'guilty']
print(classification_report(y, predict_svc, target_names = target_names))

              precision    recall  f1-score   support

  displeased       0.85      0.34      0.49        67
   depressed       0.52      0.93      0.66        88
        hurt       0.12      0.10      0.11        10
      lonely       0.00      0.00      0.00         6
       grief       0.00      0.00      0.00        12
      guilty       0.00      0.00      0.00        12

    accuracy                           0.54       195
   macro avg       0.25      0.23      0.21       195
weighted avg       0.53      0.54      0.47       195



/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [57]:
## Prediction
sentence = ["she has depression"]
X_test1 = vectorizer.transform(sentence)
#X_test1

dtree_model.fit(X, y)
dtree_model.predict(X_test1.todense())

/Users/keemia/opt/miniconda3/envs/env_419/lib/python3.10/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


array(['depressed'], dtype=object)